In [79]:
from maap.maap import MAAP
# maap = MAAP(maap_host='api.maap-project.org')
maap = MAAP()

maap._MAAP_HOST

'api.maap-project.org'

### Use MAAP Registration call in notebook chunk to register DPS algorithm
 - We need to register a DPS algorithm called 'do_HLS_stack_3-1-2' before proceeding to the chunks below...

In [106]:
maap.register_algorithm_from_yaml_file("/projects/icesat2_boreal/dps/registered/run_boreal_biomass.yml").text

'{"code": 200, "message": {"id": "781cefe85445ab5a0813548b994cdd4ef276a911", "short_id": "781cefe8", "created_at": "2023-11-22T03:25:21.000+00:00", "parent_ids": ["39f971416bd1acc704ac0cb97ea718f27a018888"], "title": "Registering algorithm: run_boreal_biomass_map", "message": "Registering algorithm: run_boreal_biomass_map", "author_name": "root", "author_email": "root@e49d5cea4b76", "authored_date": "2023-11-22T03:25:21.000+00:00", "committer_name": "root", "committer_email": "root@e49d5cea4b76", "committed_date": "2023-11-22T03:25:21.000+00:00", "trailers": {}, "web_url": "https://repo.maap-project.org/root/register-job-hysds-v4/-/commit/781cefe85445ab5a0813548b994cdd4ef276a911", "stats": {"additions": 0, "deletions": 0, "total": 0}, "status": "pending", "project_id": 3, "last_pipeline": {"id": 9904, "iid": 537, "project_id": 3, "sha": "781cefe85445ab5a0813548b994cdd4ef276a911", "ref": "main", "status": "pending", "source": "push", "created_at": "2023-11-22T03:25:22.891Z", "updated_at

# Launch DPS for mapBoreal.R

In [57]:
import os
import geopandas
import pandas as pd
import json
import datetime
#!pip install xmltodict
import xmltodict

def local_to_s3(url, user='lduncanson'):
    ''' A Function to convert local paths to s3 urls'''
    return url.replace('/projects/my-private-bucket', f's3://maap-ops-workspace/{user}')
def local_to_https(url, user='lduncanson'):
    ''' A Function to convert local paths to https urls'''
    return url.replace('/projects/my-private-bucket', f'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/{user}')
def local_to_https_uswest2(url, user='lduncanson'):
    ''' A Function to convert local paths to https us-west-s urls'''
    return url.replace('/projects/my-private-bucket', f'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/{user}')

In [58]:
#atl08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/lduncanson/dps_output/run_tile_atl08_ubuntu/master/2022/03/ATL08_filt_tindex_master.csv')

#atl08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/ATL08_filt_tindex_master.csv')
#my-public-bucket/DPS_tile_lists/fall2022/ATL08_filt_tindex_master.csv

#note - check that the atl08 version matched ground = with_gedi_rh noground = with_atl03_rh
atl08_filt_tindex_master =   pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/ATL08_filt/c2020/tile_atl08/ATL08_filt_tindex_master.csv')
topo_tindex_master =         pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv')
landsat_tindex_master =      pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/HLS/c2020/HLS_stack_2022_v2/HLS_tindex_master.csv')
lc_tindex_master = pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/LC/LC_tindex_master.csv')



# Convert al local_paths to s3
#.. for data produced by 'lduncanson' workspace
atl08_filt_tindex_master['https'] = [local_to_https_uswest2(local_path, user='lduncanson') for local_path in atl08_filt_tindex_master['local_path']]

#.. for data produced by 'nathanmthomas' workspace
for tindex_master in [topo_tindex_master, landsat_tindex_master, lc_tindex_master]:
    tindex_master['https'] = [local_to_https_uswest2(local_path, user='nathanmthomas') for local_path in tindex_master['local_path']]

# Use the ATL08 filtered tindex master list to tell you which tiles you'll run

In [98]:

NORWAY_TILE_LIST = pd.read_csv('/projects/my-public-bucket/misc_files/norway_tiles.csv').layer.tolist()

#NORWAY_TILE_LIST = [42,199,200,223,224,386,387,11,49,111,112,131]

DELTA_TILE_LIST = [3459]

BONA_TILE_LIST  = [3364,3365]

HEALY_TILE_LIST = [3552]

INPUT_EXPERIMENT_TILE_NUM_LIST = NORWAY_TILE_LIST + DELTA_TILE_LIST + BONA_TILE_LIST + HEALY_TILE_LIST
#DPS_INPUT_TILE_NUM_LIST = INPUT_EXPERIMENT_TILE_NUM_LIST
#len(DPS_INPUT_TILE_NUM_LIST)

#ALASKA_TILE_LIST =  list(range(3268,3272+1))+\
#                    list(range(3361,3366+1))+\
#                    list(range(3454,3459+1))+\
#                    list(range(3549,3555+1))+\
#                    list(range(3643,3648+1))

print(len(INPUT_EXPERIMENT_TILE_NUM_LIST))
DPS_INPUT_TILE_NUM_LIST = INPUT_EXPERIMENT_TILE_NUM_LIST

FULL_TILE_LIST = atl08_filt_tindex_master['tile_num']

DPS_INPUT_TILE_NUM_LIST = [639, 640, 726, 727, 373, 405]

DPS_INPUT_TILE_NUM_LIST = FULL_TILE_LIST
DPS_INPUT_TILE_NUM_LIST = [3082,2918, 1854, 1789, 1262,34210, 33975,37,148,169,446,162,629,1761, 2651, 2970, 3604, 4080]
DPS_INPUT_TILE_NUM_LIST = [1050, 1105, 34210, 4245, 24959, 23828]

MISSING_TILE_LIST = [1216, 3653, 4151, 3827, 40664, 2745, 1321, 1326, 1318, 611, 577, 976, 1316, 36019, 36238, 36237, 36685, 39464, 4418, 4308, 4024, 4107, 4104, 4029, 2505, 3417, 41809, 42558, 24113, 24394, 24674, 26341, 26340, 25184, 40239, 39832, 36401, 35285, 35059, 35060, 34834, 34606, 3402, 3215, 3035, 2782, 1278, 1108, 26554, 10, 21, 155, 250, 357, 448, 484, 608, 609, 650, 738, 34607, 35955, 37080, 39630, 4150, 4141, 4366, 3978]
DPS_INPUT_TILE_NUM_LIST = INPUT_EXPERIMENT_TILE_NUM_LIST

DPS_INPUT_TILE_NUM_LIST = FULL_TILE_LIST


print(len(DPS_INPUT_TILE_NUM_LIST))



127
4942


In [89]:
# Check retrieval of s3 path with a tle_num
in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_lc_https = lc_tindex_master['https'].loc[lc_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]

print(in_atl08_https)
print(in_topo_https)
print(in_landsat_https)
print(in_lc_https)

https://maap-ops-workspace.s3.us-west-2.amazonaws.com/lduncanson/dps_output/run_tile_atl08_ubuntu/tile_atl08/2022/11/30/16/09/11/296408/atl08_005_30m_filt_topo_landsat_20221130_0131.csv
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/do_topo_stack_3-1-5_ubuntu/ops/2021/09/15/18/37/26/218643/Copernicus_131_covars_cog_topo_stack.tif
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/do_HLS_stack_3-1-2_ubuntu/HLS_stack_2022_v2/2022/10/12/20/45/20/047641/HLS_131_06-01_09-15_2019_2021.tif
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/run_build_stack_ubuntu/master/2022/09/15/17/23/42/788257/esa_worldcover_v100_2020_131_cog.tif


## Get files for boreal biomass models & boreal wide sample
Note that this no longer sends the bio models to the job - this is embedded in the .sh run script (not in the param dict)

In [90]:
bio_models_https = 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/lduncanson/my-private-bucket/bio_models_noground.tar'

train_data_https = 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/lduncanson/boreal_train_data_v10.csv'

In [91]:
bio_models_https

'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/lduncanson/my-private-bucket/bio_models_noground.tar'

## Test from Terminal

In [25]:
# put latest test data in a test folder (outside repo)
# Terminal:
                           
# file 1: /projects/testing/input/atl08_004_30m_filt_topo_landsat_20220208_0042.csv
# file 2: /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif
# file 3: /projects/shared-buckets/nathanmthomas/alg_34_testing/Landsat8_42_comp_cog_2015-2020_dps.tif
#file 4: /projects/my-private-bucket/boreal_train_data_v3.csv
#file 5: /projects/my-private-bucket/bio_models.tar
#file 6: 42
# file 7:

# ./run_boreal_biomass.sh file1 file2 file3 file4 file5 file6 file7
#ATL08_CSV= /projects/testing/input/atl08_005_30m_filt_topo_landsat_20220322_0042.csv
#TOPO_TIF=/projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif
#LANDSAT_TIF= /projects/shared-buckets/nathanmthomas/alg_34_testing/Landsat8_42_comp_cog_2015-2020_dps.tif
#DO_SLOPE_VALID_MASK= 'TRUE'
#ATL08_SAMPLE_CSV=/projects/my-private-bucket/boreal_train_data_v3.csv
#in_tile_num=42
#in_tile_fn=/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg

#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 42 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/
#echo $cmd
#eval $cmd

# Set the output merged CSV name to a var
#MERGED_ATL08_CSV=$(ls ${OUTPUTDIR}/atl08_005_30m_filt_merge_neighbors* | head -1)

# Run mapBoreal with merged CSV as input
#conda activate r-with-gdal


#Rscript /projects/icesat2_boreal/lib/mapBoreal.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_42_06-01_09-15_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v4.csv 2 2 100 300 5 TRUE 50 3000 

#test swapping for terra
#Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_42_06-01_09-15_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v4.csv 3 2 100 300 5 TRUE 100 3000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 

# of tiles to run: 115
# [3585, 4108, 1037, 28176, 4112, 31762, 1050, 2075, 28702, 31263, 29223, 29739, 30252, 3117, 30763, 3120, 2614, 2618, 2619, 1615, 1105, 1617, 2142, 612, 2148, 2149, 2151, 2152, 2153, 1146, 1662, 1159, 1672, 2698, 34443, 26280, 2217, 2223, 2224, 33975, 1721, 3770, 3771, 1214, 1731, 1733, 26827, 2779, 3301, 3302, 27368, 745, 3307, 1273, 2301, 27908, 1798, 1799, 1800, 3861, 28440, 3865, 31513, 28963, 31014, 29481, 2859, 29996, 30508, 1334, 4411, 3393, 3395, 2375, 3399, 1865, 1866, 1867, 2378, 2379, 2380, 2381, 3400, 3951, 3952, 34674, 3954, 2942, 895, 2944, 1933, 1934, 1937, 1938, 2457, 3486, 34210, 931, 1448, 26554, 4034, 4035, 1999, 981, 3029, 3030, 2009, 2010, 2011, 27098, 991, 992, 993, 2533, 27638]

In [26]:
# test of tile 1927
#ATL08_CSV= /projects/testing/input/atl08_005_30m_filt_topo_landsat_20220418_1927.csv
#TOPO_TIF=/projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_1927_covars_cog_topo_stack.tif 
#LANDSAT_TIF= /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_1927_06-15_09-01_2019_2021.tif
#DO_SLOPE_VALID_MASK= 'TRUE'
#ATL08_SAMPLE_CSV=/projects/my-private-bucket/boreal_train_data_v4.csv
#in_tile_num=42
#in_tile_fn=/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg

#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 1927 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn /projects/my-public-bucket/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/
#echo $cmd
#eval $cmd

# Set the output merged CSV name to a var
#MERGED_ATL08_CSV=$(ls ${OUTPUTDIR}/atl08_005_30m_filt_merge_neighbors* | head -1)

# Run mapBoreal with merged CSV as input
#conda activate r-with-gdal


#Rscript /projects/icesat2_boreal/lib/mapBoreal.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_726_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v5.csv 2 2 100 300 5 TRUE 50 3000 

#test swapping for terra
#Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/shared-buckets/nathanmthomas/atl08_004_30m_filt_merge_neighbors_3885.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_3585_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_727_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v5.csv 2 2 100 300 5 TRUE 100 3000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 



In [27]:
#test on tile 727 and 726
#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 727 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/
       
#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 726 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/    

## Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0726.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_726_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_726_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v7.csv 2 2 130 250 5 TRUE 50 5000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 

## Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0727.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_727_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_727_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v7.csv 2 2 130 250 5 TRUE 50 5000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 



## Commit with Tag for running
1) Add version name as a map_boreal_2022_v2 or whatever is appropriate - both to this notebook and algorithm config yaml

2) follow git instructions (every time!!):
git add changes
git commit -m 'message'
git tag -f map_boreal_2022_rh_noground_v4

git push
git push origin -f map_boreal_2022_rh_noground_v4
git push dps
git push dps -f map_boreal_2022_rh_noground_v4

3) if it looks weird check git log to make sure tag is at same place as origin and dps


## register via terminal
## IGNORE HAS BEEN REPLACED ABOVE
python /projects/register-algorithm.py /projects/icesat2_boreal/dps/alg_3-4/algorithm_config.yaml

## Run a DPS job


In [ ]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):

    DPS_num = i+1
    #using git tag as identifier
    IDENTIFIER = 'AGB_H30_2020/local_training_full'
    ALGO_ID = "run_boreal_biomass_map"
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-16gb'
    
    # Maybe we're using an input tile list that isnt built directly from an ATL08_filt CSV
    # so, check if we have filtered ATL08 for this tile:
    if INPUT_TILE_NUM in atl08_filt_tindex_master['tile_num'].to_list():
        
        # Get the s3 paths of the corresponding input filenames with an input tile_num
        in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_lc_https = lc_tindex_master['https'].loc[lc_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
 
    else:
        print(f"No filtered ATL08 for tile {INPUT_TILE_NUM}. Moving on.." )
        continue
    
    # iters is the number of iterations of RH model fits; For no uncertainties set iters = 1 for quick mapping / testing.
    # anything >1 will be exhaustive until uncertainties constrain or iters becomes 250. Will take a long time.
        
    # ppside is the number of subtile splits. 2=4 subtiles, 3=9 subtiles
    # increasing ppside reduces memory, increases length to completion
    
    # minDOY is the min possible DOY to search for tile training data. The default is set to May 1
    # If sufficient data are found in growing season, minDOY will not be used
    
    # maxDOY is the max possible DOY to search for tile training data. The default is Sept 30.
    
    # max_sol_el is the max solar elevation possible to search. The default is 0
    
    # expand_training as TRUE sets iterative searching for training data up to min_n, starting with solar_el<0 and growing season and then:
    # searches solar_el up to max_sol_el then
    # searches iteratively one month later after growing season until maxDOY then
    # searches iteratively one month earlier before growing season until minDOY
    # if expand_training=FALSE the growing season sol_el<0 will be taken, and if none, no data will return
    
    # min_n is the tile training number desired, that expand_training will search until it is fulfilled (or the max available)
    
    #local_train_perc is the percent tile training data vs. boreal-wide training data. 100 = all local, 0 = all boreal wide
        
    in_param_dict = {
                                    'in_atl08_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/ATL08_filt/c2020/tile_atl08/ATL08_filt_tindex_master.csv',
                                    'in_topo_fn': f"input/{os.path.basename(in_topo_https)}",
                                    'in_landsat_fn': f"input/{os.path.basename(in_landsat_https)}",
                                    'in_lc_fn': f"input/{os.path.basename(in_lc_https)}",
                                    'in_atl08_fn_url': in_atl08_https,
                                    'in_topo_fn_url': in_topo_https,
                                    'in_landsat_fn_url': in_landsat_https,
                                    'in_lc_fn_url': in_lc_https,
                                    'DO_SLOPE_VALID_MASK': 'TRUE',
                                    'in_atl08_sample_fn':f"input/{os.path.basename(train_data_https)}",
                                    'in_atl08_sample_url':train_data_https,
                                    'in_tile_num': INPUT_TILE_NUM,
                                    'in_tile_fn_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
                                    'in_tile_fn': 'input/boreal_tiles_v003.gpkg',
                                    'in_tile_field': 'tile_num',
                                    'iters': 1,
                                    'ppside': 2,
                                    'minDOY':130,
                                    'maxDOY':250,
                                    'max_sol_el':5,
                                    'expand_training': 'TRUE',
                                    'local_train_perc': 100,
                                    'min_n': 5000, 
                                    'boreal_vect': 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/shared/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson',
                                    'boreal_vect_fn': 'input/wwf_circumboreal_Dissolve_reprj.geojson',
                                    'predict_var': 'AGB'
                }
    
    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='boreal_agb_2023_v3',#, ht_boreal_2022_rh_noground_v1 ,'map_boreal_2022_rh_ground_v1','map_boreal_2022_rh_noground_v1',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )


# of input tiles for DPS: 4938


In [16]:
   # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_TILE_NUM
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE

    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 2, 25, 50, 100, 500, 1000, 1500, 2000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 
        #local_path

# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df

IndentationError: unexpected indent (2631151225.py, line 2)

## Assess DPS Results

In [19]:
%%time
#submit_results_df = pd.read_csv('/projects/my-public-bucket/DPS_run_boreal_biomass_quick_v2_submission_results_68_202302150220.csv')
submit_results_df = pd.concat(submit_results_df_list)

def BUILD_TABLE_JOBSTATUS(submit_results_df):
    import xmltodict
    df = pd.concat([pd.DataFrame(xmltodict.parse(maap.getJobStatus(job_id).content)).transpose() for job_id in submit_results_df.job_id.to_list()])
    return df

job_status_df = BUILD_TABLE_JOBSTATUS(submit_results_df)
#print(job_status_df.head())

num_jobs = submit_results_df.shape[0]
z = submit_results_df.merge(job_status_df, how='left', left_on='job_id',  right_on='wps:JobID')

print(f'Count total jobs:\t{num_jobs}')
print(f"Count pending jobs:\t{z[z['wps:Status'] =='Accepted'].shape[0]}")
print(f"Count running jobs:\t{z[z['wps:Status'] =='Running'].shape[0]}")
print(f"Count succeeded jobs:\t{z[z['wps:Status'] =='Succeeded'].shape[0]}")
print(f"Count failed jobs:\t{z[z['wps:Status'] =='Failed'].shape[0]}")
print(f"% of failed jobs:\t{round(z[z['wps:Status'] =='Failed'].shape[0] / ( z[z['wps:Status'] =='Failed'].shape[0] + z[z['wps:Status'] =='Succeeded'].shape[0] ), 4) * 100}")

Count total jobs:	15
Count pending jobs:	0
Count running jobs:	7
Count succeeded jobs:	0
Count failed jobs:	8
% of failed jobs:	100.0
CPU times: user 177 ms, sys: 14.1 ms, total: 191 ms
Wall time: 394 ms


In [41]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Failed'].iloc[0].job_id).content)

{'wps:Result': {'@xmlns:ows': 'http://www.opengis.net/ows/2.0',
  '@xmlns:schemaLocation': 'http://schemas.opengis.net/wps/2.0/wps.xsd',
  '@xmlns:wps': 'http://www.opengis.net/wps/2.0',
  '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  'wps:JobID': '9b9e1479-faf2-4093-b1ca-03fb2dbe1bed',
  'wps:Output': {'@id': 'traceback',
   'wps:Data': 'Traceback (most recent call last):\n  File "/home/ops/verdi/ops/hysds-0.3.11/hysds/job_worker.py", line 1126, in run_job\n    raise RuntimeError("Got non-zero exit code: {}".format(status))\nRuntimeError: Got non-zero exit code: 255'}}}

In [136]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Failed'].iloc[20].job_id).content)

{'wps:Result': {'@xmlns:ows': 'http://www.opengis.net/ows/2.0',
  '@xmlns:schemaLocation': 'http://schemas.opengis.net/wps/2.0/wps.xsd',
  '@xmlns:wps': 'http://www.opengis.net/wps/2.0',
  '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  'wps:JobID': '167d1e36-6d6e-4d4c-9549-b02dae4d038e',
  'wps:Output': {'@id': 'traceback',
   'wps:Data': 'Traceback (most recent call last):\n  File "/home/ops/verdi/ops/hysds-0.3.11/hysds/job_worker.py", line 1085, in run_job\n    monitoredRunner.join()\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/process.py", line 148, in join\n    res = self._popen.wait(timeout)\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/popen_fork.py", line 57, in wait\n    return self.poll(os.WNOHANG if timeout == 0.0 else 0)\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/popen_fork.py", line 33, in poll\n    pid, sts = os.waitpid(self.pid, flag)\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/pool.py", l

## After successful DPS run, create AGB tindex

In [ ]:
## create AGB tindex 
USER = 'lduncanson'
month_dir_str = '11'
DPS_MONTH_LIST = '01 02 03 04 05 06 07 08 09 10 11 12'
alg_name_str = 'run_boreal_biomass_map'
DPS_IDENTIFIER = 'boreal_agb_2023_v3/AGB_H30_2020/local_training_full'
#index_out_dir = os.path.join('/projects/my-private-bucket/dps_output/run_boreal_biomass_qiuck_v2_ubuntu/map_boreal_2022_v3/2022/', month_dir_str)
#index_out_dir = '/projects/my-public-bucket/DPS_tile_lists/2023/'
OUTDIR = f'/projects/my-public-bucket/DPS_tile_lists/BOREAL_MAP/{DPS_IDENTIFIER}'
!mkdir -p $OUTDIR

!python /projects/icesat2_boreal/lib/build_tindex_master.py -t 'AGB' -alg_name $alg_name_str --dps_identifier $DPS_IDENTIFIER -y 2023 -m_list $DPS_MONTH_LIST -o $OUTDIR




In [101]:
TEST_DPS  = True

if TEST_DPS:
    
    if True:
        #!python /projects/icesat2_boreal/lib/build_tindex_master_v2.py -t AGB -y 2022 -m $month_dir_str -o $index_out_dir
        t = pd.read_csv(os.path.join(OUTDIR,'AGB_tindex_master.csv'))
        COMPLETED_TILES = t.tile_num.to_list()
        NEED_TILES = list(set(DPS_INPUT_TILE_NUM_LIST) - set(COMPLETED_TILES))

        #print(NEED_TILES)
        DPS_INPUT_TILE_NUM_LIST = NEED_TILES
    
else:
    DPS_INPUT_TILE_NUM_LIST = INPUT_TILE_NUM_LIST
   

#DPS_INPUT_TILE_NUM_LIST = [248, 273, 272, 271, 324]
print(f"# of tiles to run: {len(DPS_INPUT_TILE_NUM_LIST)}\n", DPS_INPUT_TILE_NUM_LIST)


# of tiles to run: 4938
 [4, 36471, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 29, 30, 31, 36472, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 36475, 108, 109, 110, 111, 112, 113, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 195, 196, 197, 198, 199, 200, 201, 203, 204, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240

In [105]:
t

,Unnamed: 0,s3_path,local_path,file,tile_num,creation time
0,3,s3://maap-ops-workspace/lduncanson/dps_output/...,/projects/my-private-bucket/dps_output/run_bor...,boreal_agb_202311211700621578_0205.tif,205,2023-11-21 18:54:06
1,2,s3://maap-ops-workspace/lduncanson/dps_output/...,/projects/my-private-bucket/dps_output/run_bor...,boreal_agb_202311211700621068_1289.tif,1289,2023-11-21 18:46:07
2,1,s3://maap-ops-workspace/lduncanson/dps_output/...,/projects/my-private-bucket/dps_output/run_bor...,boreal_agb_202311211700619873_4501.tif,4501,2023-11-21 18:25:42
3,0,s3://maap-ops-workspace/lduncanson/dps_output/...,/projects/my-private-bucket/dps_output/run_bor...,boreal_agb_202311211700618782_4418.tif,4418,2023-11-21 18:06:36


## Re-run Failed Tiles in DPS

In [18]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):

    DPS_num = i+1
    IDENTIFIER = 'run_boreal_biomass_quick_v2'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-16gb'
    
    # Maybe we're using an input tile list that isnt built directly from an ATL08_filt CSV
    # so, check if we have filtered ATL08 for this tile:
    if INPUT_TILE_NUM in atl08_filt_tindex_master['tile_num'].to_list():
        
        # Get the s3 paths of the corresponding input filenames with an input tile_num
        in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_lc_https = lc_tindex_master['https'].loc[lc_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
 
    else:
        print(f"No filtered ATL08 for tile {INPUT_TILE_NUM}. Moving on.." )
        continue
    
    # iters is the number of iterations of RH model fits; set to low (2-5) to test, 30-50 for production
    # 30 yields run times ~4 hours
        
    # ppside is the number of subtile splits. 2=4 subtiles, 3=9 subtiles
    # increasing ppside reduces memory, increases length to completion
    
    # minDOY is the min possible DOY to search for tile training data. The default is set to May 1
    # If sufficient data are found in growing season, minDOY will not be used
    
    # maxDOY is the max possible DOY to search for tile training data. The default is Sept 30.
    
    # max_sol_el is the max solar elevation possible to search. The default is 0
    
    # expand_training as TRUE sets iterative searching for training data up to min_n, starting with solar_el<0 and growing season and then:
    # searches solar_el up to max_sol_el then
    # searches iteratively one month later after growing season until maxDOY then
    # searches iteratively one month earlier before growing season until minDOY
    # if expand_training=FALSE the growing season sol_el<0 will be taken, and if none, no data will return
    
    # min_n is the tile training number desired, that expand_training will search until it is fulfilled (or the max available)
    
    #local_train_perc is the percent tile training data vs. boreal-wide training data. 100 = all local, 0 = all boreal wide
    
    
    in_param_dict = {
                                    'in_atl08_fn': 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/fall2022/ATL08_filt_tindex_master.csv',
                                    #'in_atl08_fn': f"input/{os.path.basename(in_atl08_https)}",
                                    'in_topo_fn': f"input/{os.path.basename(in_topo_https)}",
                                    'in_landsat_fn': f"input/{os.path.basename(in_landsat_https)}",
                                    'in_lc_fn': f"input/{os.path.basename(in_lc_https)}",
                                    'in_atl08_fn_url': in_atl08_https,
                                    'in_topo_fn_url': in_topo_https,
                                    'in_landsat_fn_url': in_landsat_https,
                                    'in_lc_fn_url': in_lc_https,
                                    'DO_SLOPE_VALID_MASK': 'TRUE',
                                    'in_atl08_sample_fn':f"input/{os.path.basename(train_data_https)}",
                                    'in_atl08_sample_url':train_data_https,
                                    'in_tile_num': INPUT_TILE_NUM,
                                    'in_tile_fn_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
                                    'in_tile_fn': 'input/boreal_tiles_v003.gpkg',
                                    'in_tile_field': 'tile_num',
                                    'iters': 1,
                                    'ppside': 2,
                                    'minDOY':130,
                                    'maxDOY':250,
                                    'max_sol_el':5,
                                    'expand_training': 'TRUE',
                                    'local_train_perc': 100,
                                    'min_n': 5000, 
                                    'boreal_vect': 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/shared/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson',
                                    'boreal_vect_fn': 'input/wwf_circumboreal_Dissolve_reprj.geojson'
                }
    
    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='map_boreal_2022_v3',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )

    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_TILE_NUM
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE

    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 2, 25, 50, 100, 500, 1000, 1500, 2000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 
        #local_path

# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df

# of input tiles for DPS: 219
DPS run #: 1	| tile num: 1539	| submit status: success	| job id: 24a8117e-1e63-4762-8007-a71545ae36e2
DPS run #: 2	| tile num: 3078	| submit status: success	| job id: ba3efa1c-f4c9-4f86-80a0-e17b5a1d2c07
DPS run #: 25	| tile num: 3633	| submit status: success	| job id: a5a7b136-c7bc-4ddd-a28f-aca8a01f96b9
DPS run #: 50	| tile num: 3178	| submit status: success	| job id: 9a772e83-2864-495b-8758-f79780c04abf
DPS run #: 100	| tile num: 2265	| submit status: success	| job id: 695d01d8-d6dd-4bab-922f-f0dd2e15e30f
Current time:	202211210208


ImportError: cannot import name 'CompressionOptions' from 'pandas._typing' (/opt/conda/lib/python3.7/site-packages/pandas/_typing.py)

## Re-Assess Re-submission
Continue with last few steps until all jobs successful: create tindex, create new tile list, re-run missing tiles in DPS, re-assess

In [36]:
%%time
#submit_results_df = pd.read_csv('/projects/my-public-bucket/DPS_run_boreal_biomass_quick_v2_submission_results_2494_202211132259.csv')
submit_results_df = pd.concat(submit_results_df_list)

def BUILD_TABLE_JOBSTATUS(submit_results_df):
    import xmltodict
    df = pd.concat([pd.DataFrame(xmltodict.parse(maap.getJobStatus(job_id).content)).transpose() for job_id in submit_results_df.job_id.to_list()])
    return df

job_status_df = BUILD_TABLE_JOBSTATUS(submit_results_df)
#print(job_status_df.head())

num_jobs = submit_results_df.shape[0]
z = submit_results_df.merge(job_status_df, how='left', left_on='job_id',  right_on='wps:JobID')

print(f'Count total jobs:\t{num_jobs}')
print(f"Count pending jobs:\t{z[z['wps:Status'] =='Accepted'].shape[0]}")
print(f"Count running jobs:\t{z[z['wps:Status'] =='Running'].shape[0]}")
print(f"Count succeeded jobs:\t{z[z['wps:Status'] =='Succeeded'].shape[0]}")
print(f"Count failed jobs:\t{z[z['wps:Status'] =='Failed'].shape[0]}")
print(f"% of failed jobs:\t{round(z[z['wps:Status'] =='Failed'].shape[0] / ( z[z['wps:Status'] =='Failed'].shape[0] + z[z['wps:Status'] =='Succeeded'].shape[0] ), 4) * 100}")

Count total jobs:	219
Count pending jobs:	0
Count running jobs:	70
Count succeeded jobs:	15
Count failed jobs:	134
% of failed jobs:	89.92999999999999
CPU times: user 2.24 s, sys: 197 ms, total: 2.44 s
Wall time: 5.08 s


## Get other lists just of missing tiles (various ways)

In [22]:
need_tindex_master = pd.read_csv('/projects/my-public-bucket/DPS_tile_lists/Need_AGB_tindex_master.csv')
print(len(need_tindex_master))

INPUT_TILE_NUM_LIST = need_tindex_master.tile_num.tolist()

# Remove duplicate tile nums
INPUT_TILE_NUM_LIST = list(set(INPUT_TILE_NUM_LIST))

print(len(INPUT_TILE_NUM_LIST))

INPUT_TILE_NUM_LIST_NEED = INPUT_TILE_NUM_LIST

1
1


In [9]:
weird_tiles = [4304, 4305, 4221, 4220, 1785, 1718, 1720, 1661, 1257, 1318, 1317]
werid_tiles = [1255, 1196, 949, 1062, 1063, 1005, 950, 1004]
INPUT_TILE_NUM_LIST = weird_tiles
len(INPUT_TILE_NUM_LIST)

11

In [10]:
# Get all boreal tiles
ATL08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv')
boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'
boreal_tile_index = geopandas.read_file(boreal_tile_index_path)
boreal_tile_index.rename(columns={"layer":"tile_num"}, inplace=True)
boreal_tile_index["tile_num"] = boreal_tile_index["tile_num"].astype(int)

bad_tiles = [3540,3634,3728,3823,3916,4004] #Dropping the tiles near antimeridian that reproject poorly.
select_needs = [3360,2994,3190,2840,3012,3014,3017,2932,1261,1263,1264,988,978,794, 380,378,411,821,861,
                812,765,764,1308,1302,1469,1406,2495,2883,2965,3321,3509,3510,3327,3335,2976,2906,2907,2894,2814,4253,4293,4403,4440,4408,4372,4477,3986]

# Remove bad tiles
boreal_tile_index = boreal_tile_index[~boreal_tile_index['tile_num'].isin(bad_tiles)]
    
#print(boreal_tile_index.head())
tile_matches_select_needs = boreal_tile_index.merge(ATL08_filt_tindex_master[ATL08_filt_tindex_master['tile_num'].isin(select_needs)], how='right', on='tile_num')
print(len(tile_matches_select_needs))
select_needs_filt = print([t for t in tile_matches_select_needs.tile_num])

45
[978, 988, 978, 988, 1263, 1261, 1264, 794, 2932, 3190, 3012, 3014, 3360, 3017, 378, 765, 812, 821, 764, 861, 1302, 1308, 1469, 2894, 2883, 2965, 2976, 3327, 3321, 3510, 3509, 4372, 4440, 4477, 2994, 1406, 2840, 411, 380, 3335, 2495, 4403, 2906, 2907, 2814]


In [32]:
#select_needs_filt = [978, 988, 978, 988, 1263, 1261, 1264, 794, 2932, 3190, 3012, 3014, 3360, 3017, 378, 765, 812, 821, 764, 861, 1302, 1308, 1469, 2894, 2883, 2965, 2976, 3327, 3321, 3510, 3509, 4372, 4440, 4477, 2994, 1406, 2840, 411, 380, 3335, 2495, 4403, 2906, 2907, 2814]

print(len(select_needs_filt))

import numpy as np
tile_nums_missing_but_wont_run = np.setdiff1d(select_needs, select_needs_filt)
print(len(tile_nums_missing_but_wont_run))
print(tile_nums_missing_but_wont_run)

tile_nums_missing_periphery = np.setdiff1d(INPUT_TILE_NUM_LIST_NEED, select_needs_filt)
print(len(tile_nums_missing_periphery))
print(tile_nums_missing_periphery)
INPUT_TILE_NUM_LIST = tile_nums_missing_periphery
len(INPUT_TILE_NUM_LIST)    

45
4
[3986 4253 4293 4408]
70
[  65  106  127  250  361  417  493  566  568  618  741  753  786  798
  801  830  848  876  877  927 1273 1287 1318 1402 1438 1439 1440 1604
 1848 1850 1852 1936 2425 2426 2952 3124 3355 3540 3542 3634 3728 3823
 3845 3846 3895 3916 4004 4007 4018 4083 4176 4204 4250 4256 4307 4325
 4342 4395 4417 4444 4450 4451 4463 4467 4468 4483 4499 4510 4527 4534]


70

In [18]:
INPUT_TILE_NUM_LIST = [4440,4372,4477]

## Send output cog.tif tiles somewhere

In [12]:
import glob
out_files = glob.glob("/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/**/*cog.tif", recursive=True)

In [13]:
print(out_files)

['/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/02/23/49/35/272239/boreal_agb_20220202_0200_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/01/42/37/290043/boreal_agb_20220203_0224_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/01/52/46/430279/boreal_agb_20220203_0387_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/02/21/50/348844/boreal_agb_20220203_0386_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/01/42/237812/boreal_agb_20220203_0199_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/06/47/336472/boreal_agb_20220203_0223_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/08/52/15295

In [ ]:
TEST_SUBDIR = 'add_solar'
OUTDIR = f'/projects/my-private-bucket/dps_output/run_boreal_biomass_v2_ubuntu/master/2022/{TEST_SUBDIR}'
!python /projects/icesat2_boreal/lib/build_tindex_master.py -y 2022 -m $TEST_SUBDIR -o $OUTDIR

In [2]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')
!pip install -U -r /projects/icesat2_boreal/dps/requirements_main.txt

  Using cached pandas-1.2.2-cp37-cp37m-manylinux1_x86_64.whl (9.9 MB)
  Using cached pygeos-0.12.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached importlib_resources-5.10.2-py3-none-any.whl (34 kB)
  Using cached contextily-1.3.0-py3-none-any.whl (16 kB)
  Using cached fsspec-2023.1.0-py3-none-any.whl (143 kB)
  Using cached s3fs-0.3.4-py3-none-any.whl (18 kB)
  Using cached rio_tiler-2.1.4-py3-none-any.whl
  Using cached morecantile-2.1.4-py3-none-any.whl
  Using cached pystac_client-0.5.1-py3-none-any.whl (29 kB)
  Using cached cachetools-5.3.0-py3-none-any.whl (9.3 kB)
  Using cached pystac-1.5.0-py3-none-any.whl (146 kB)
  Using cached xyzservices-2022.9.0-py3-none-any.whl (55 kB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.4
    Uninstalling pandas-1.1.4:
      Successfully uninstalled pandas-1.1.4
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


# Launch DPS for mapBoreal.R

In [3]:
import os
import geopandas
import pandas as pd
import json
import datetime
!pip install xmltodict
import xmltodict

def local_to_s3(url, user='lduncanson'):
    ''' A Function to convert local paths to s3 urls'''
    return url.replace('/projects/my-private-bucket', f's3://maap-ops-workspace/{user}')
def local_to_https(url, user='lduncanson'):
    ''' A Function to convert local paths to https urls'''
    return url.replace('/projects/my-private-bucket', f'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/{user}')
def local_to_https_uswest2(url, user='lduncanson'):
    ''' A Function to convert local paths to https us-west-s urls'''
    return url.replace('/projects/my-private-bucket', f'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/{user}')

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


  Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [9]:
#atl08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/lduncanson/dps_output/run_tile_atl08_ubuntu/master/2022/03/ATL08_filt_tindex_master.csv')

#atl08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/ATL08_filt_tindex_master.csv')
#my-public-bucket/DPS_tile_lists/fall2022/ATL08_filt_tindex_master.csv

#note - check that the atl08 version matched ground = with_gedi_rh noground = with_atl03_rh
atl08_filt_tindex_master =   pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/ATL08_filt/c2020/tile_atl08/ATL08_filt_tindex_master.csv')
topo_tindex_master =         pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv')
landsat_tindex_master =      pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/HLS/c2020/HLS_stack_2022_v2/HLS_tindex_master.csv')
lc_tindex_master = pd.read_csv('/projects/shared-buckets/nathanmthomas/DPS_tile_lists/LC/LC_tindex_master.csv')



# Convert al local_paths to s3
#.. for data produced by 'lduncanson' workspace
atl08_filt_tindex_master['https'] = [local_to_https_uswest2(local_path, user='lduncanson') for local_path in atl08_filt_tindex_master['local_path']]

#.. for data produced by 'nathanmthomas' workspace
for tindex_master in [topo_tindex_master, landsat_tindex_master, lc_tindex_master]:
    tindex_master['https'] = [local_to_https_uswest2(local_path, user='nathanmthomas') for local_path in tindex_master['local_path']]

# Use the ATL08 filtered tindex master list to tell you which tiles you'll run

In [10]:

NORWAY_TILE_LIST = pd.read_csv('/projects/my-public-bucket/misc_files/norway_tiles.csv').layer.tolist()

#NORWAY_TILE_LIST = [42,199,200,223,224,386,387,11,49,111,112,131]

DELTA_TILE_LIST = [3459]

BONA_TILE_LIST  = [3364,3365]

HEALY_TILE_LIST = [3552]

INPUT_EXPERIMENT_TILE_NUM_LIST = NORWAY_TILE_LIST + DELTA_TILE_LIST + BONA_TILE_LIST + HEALY_TILE_LIST
#INPUT_EXPERIMENT_TILE_NUM_LIST = NORWAY_TILE_LIST
#DPS_INPUT_TILE_NUM_LIST = INPUT_EXPERIMENT_TILE_NUM_LIST
#len(DPS_INPUT_TILE_NUM_LIST)

#ALASKA_TILE_LIST =  list(range(3268,3272+1))+\
#                    list(range(3361,3366+1))+\
#                    list(range(3454,3459+1))+\
#                    list(range(3549,3555+1))+\
#                    list(range(3643,3648+1))

print(len(INPUT_EXPERIMENT_TILE_NUM_LIST))
DPS_INPUT_TILE_NUM_LIST = INPUT_EXPERIMENT_TILE_NUM_LIST

FULL_TILE_LIST = atl08_filt_tindex_master['tile_num']

DPS_INPUT_TILE_NUM_LIST = [639, 640, 726, 727, 373, 405]

DPS_INPUT_TILE_NUM_LIST = FULL_TILE_LIST
DPS_INPUT_TILE_NUM_LIST = [3082,2918, 1854, 1789, 1262,34210, 33975,37,148,169,446,162,629,1761, 2651, 2970, 3604, 4080]
DPS_INPUT_TILE_NUM_LIST = [1050, 1105, 34210, 4245, 24959, 23828]

MISSING_TILE_LIST = [1216, 3653, 4151, 3827, 40664, 2745, 1321, 1326, 1318, 611, 577, 976, 1316, 36019, 36238, 36237, 36685, 39464, 4418, 4308, 4024, 4107, 4104, 4029, 2505, 3417, 41809, 42558, 24113, 24394, 24674, 26341, 26340, 25184, 40239, 39832, 36401, 35285, 35059, 35060, 34834, 34606, 3402, 3215, 3035, 2782, 1278, 1108, 26554, 10, 21, 155, 250, 357, 448, 484, 608, 609, 650, 738, 34607, 35955, 37080, 39630, 4150, 4141, 4366, 3978]
DPS_INPUT_TILE_NUM_LIST = INPUT_EXPERIMENT_TILE_NUM_LIST

DPS_INPUT_TILE_NUM_LIST = MISSING_TILE_LIST


print(len(DPS_INPUT_TILE_NUM_LIST))



127
68


In [11]:
# Check retrieval of s3 path with a tle_num
in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]
in_lc_https = lc_tindex_master['https'].loc[lc_tindex_master['tile_num'] == DPS_INPUT_TILE_NUM_LIST[0]].tolist()[0]

print(in_atl08_https)
print(in_topo_https)
print(in_landsat_https)
print(in_lc_https)

https://maap-ops-workspace.s3.us-west-2.amazonaws.com/lduncanson/dps_output/run_tile_atl08_ubuntu/tile_atl08/2022/11/30/19/22/04/120959/atl08_005_30m_filt_topo_landsat_20221130_1216.csv
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/do_topo_stack_3-1-5_ubuntu/ops/2021/09/16/02/40/29/871367/Copernicus_1216_covars_cog_topo_stack.tif
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/do_HLS_stack_3-1-2_ubuntu/HLS_stack_2022_v2/2023/02/09/13/33/25/149108/HLS_1216_06-15_09-01_2019_2021.tif
https://maap-ops-workspace.s3.us-west-2.amazonaws.com/nathanmthomas/dps_output/run_build_stack_ubuntu/master/2022/09/15/16/43/57/807539/esa_worldcover_v100_2020_1216_cog.tif


## Get files for boreal biomass models & boreal wide sample
Note that this no longer sends the bio models to the job - this is embedded in the .sh run script (not in the param dict)

In [12]:
bio_models_https = 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/lduncanson/my-private-bucket/bio_models_noground.tar'

train_data_https = 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/lduncanson/boreal_train_data_v10.csv'

In [13]:
bio_models_https

'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/lduncanson/my-private-bucket/bio_models_noground.tar'

## Test from Terminal

In [8]:
# put latest test data in a test folder (outside repo)
# Terminal:
                           
# file 1: /projects/testing/input/atl08_004_30m_filt_topo_landsat_20220208_0042.csv
# file 2: /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif
# file 3: /projects/shared-buckets/nathanmthomas/alg_34_testing/Landsat8_42_comp_cog_2015-2020_dps.tif
#file 4: /projects/my-private-bucket/boreal_train_data_v3.csv
#file 5: /projects/my-private-bucket/bio_models.tar
#file 6: 42
# file 7:

# ./run_boreal_biomass.sh file1 file2 file3 file4 file5 file6 file7
#ATL08_CSV= /projects/testing/input/atl08_005_30m_filt_topo_landsat_20220322_0042.csv
#TOPO_TIF=/projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif
#LANDSAT_TIF= /projects/shared-buckets/nathanmthomas/alg_34_testing/Landsat8_42_comp_cog_2015-2020_dps.tif
#DO_SLOPE_VALID_MASK= 'TRUE'
#ATL08_SAMPLE_CSV=/projects/my-private-bucket/boreal_train_data_v3.csv
#in_tile_num=42
#in_tile_fn=/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg

#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 42 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/
#echo $cmd
#eval $cmd

# Set the output merged CSV name to a var
#MERGED_ATL08_CSV=$(ls ${OUTPUTDIR}/atl08_005_30m_filt_merge_neighbors* | head -1)

# Run mapBoreal with merged CSV as input
#conda activate r-with-gdal


#Rscript /projects/icesat2_boreal/lib/mapBoreal.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_42_06-01_09-15_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v4.csv 2 2 100 300 5 TRUE 50 3000 

#test swapping for terra
#Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_42_06-01_09-15_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v4.csv 3 2 100 300 5 TRUE 100 3000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 

# of tiles to run: 115
# [3585, 4108, 1037, 28176, 4112, 31762, 1050, 2075, 28702, 31263, 29223, 29739, 30252, 3117, 30763, 3120, 2614, 2618, 2619, 1615, 1105, 1617, 2142, 612, 2148, 2149, 2151, 2152, 2153, 1146, 1662, 1159, 1672, 2698, 34443, 26280, 2217, 2223, 2224, 33975, 1721, 3770, 3771, 1214, 1731, 1733, 26827, 2779, 3301, 3302, 27368, 745, 3307, 1273, 2301, 27908, 1798, 1799, 1800, 3861, 28440, 3865, 31513, 28963, 31014, 29481, 2859, 29996, 30508, 1334, 4411, 3393, 3395, 2375, 3399, 1865, 1866, 1867, 2378, 2379, 2380, 2381, 3400, 3951, 3952, 34674, 3954, 2942, 895, 2944, 1933, 1934, 1937, 1938, 2457, 3486, 34210, 931, 1448, 26554, 4034, 4035, 1999, 981, 3029, 3030, 2009, 2010, 2011, 27098, 991, 992, 993, 2533, 27638]

In [9]:
# test of tile 1927
#ATL08_CSV= /projects/testing/input/atl08_005_30m_filt_topo_landsat_20220418_1927.csv
#TOPO_TIF=/projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_1927_covars_cog_topo_stack.tif 
#LANDSAT_TIF= /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_1927_06-15_09-01_2019_2021.tif
#DO_SLOPE_VALID_MASK= 'TRUE'
#ATL08_SAMPLE_CSV=/projects/my-private-bucket/boreal_train_data_v4.csv
#in_tile_num=42
#in_tile_fn=/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg

#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 1927 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn /projects/my-public-bucket/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/
#echo $cmd
#eval $cmd

# Set the output merged CSV name to a var
#MERGED_ATL08_CSV=$(ls ${OUTPUTDIR}/atl08_005_30m_filt_merge_neighbors* | head -1)

# Run mapBoreal with merged CSV as input
#conda activate r-with-gdal


#Rscript /projects/icesat2_boreal/lib/mapBoreal.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0042.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_42_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_726_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v5.csv 2 2 100 300 5 TRUE 50 3000 

#test swapping for terra
#Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/shared-buckets/nathanmthomas/atl08_004_30m_filt_merge_neighbors_3885.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_3585_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_727_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v5.csv 2 2 100 300 5 TRUE 100 3000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 



In [10]:
#test on tile 727 and 726
#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 727 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/
       
#python /projects/icesat2_boreal/lib/merge_neighbors_atl08.py -in_tile_num 726 -in_tile_fn /projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg -in_tile_field 'tile_num' -csv_list_fn s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv -out_dir /projects/testing/output/    

## Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0726.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_726_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_726_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v7.csv 2 2 130 250 5 TRUE 50 5000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 

## Rscript /projects/icesat2_boreal/lib/mapBoreal_speedy.R /projects/testing/output/atl08_004_30m_filt_merge_neighbors_0727.csv /projects/shared-buckets/nathanmthomas/alg_34_testing/Copernicus_727_covars_cog_topo_stack.tif /projects/shared-buckets/nathanmthomas/alg_34_testing/HLS_727_06-15_09-01_2019_2021.tif 'TRUE' /projects/my-private-bucket/boreal_train_data_v7.csv 2 2 130 250 5 TRUE 50 5000 /projects/shared-buckets/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson 



## Commit with Tag for running
1) Add version name as a map_boreal_2022_v2 or whatever is appropriate - both to this notebook and algorithm config yaml

2) follow git instructions (every time!!):
git add changes
git commit -m 'message'
git tag -f map_boreal_2022_rh_noground_v4

git push
git push origin -f map_boreal_2022_rh_noground_v4
git push dps
git push dps -f map_boreal_2022_rh_noground_v4

3) if it looks weird check git log to make sure tag is at same place as origin and dps


## register via terminal
python /projects/register-algorithm.py /projects/icesat2_boreal/dps/alg_3-4/algorithm_config.yaml

## Run a DPS job


In [15]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):

    DPS_num = i+1
    IDENTIFIER = 'run_boreal_biomass_quick_v2'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-32gb'
    
    # Maybe we're using an input tile list that isnt built directly from an ATL08_filt CSV
    # so, check if we have filtered ATL08 for this tile:
    if INPUT_TILE_NUM in atl08_filt_tindex_master['tile_num'].to_list():
        
        # Get the s3 paths of the corresponding input filenames with an input tile_num
        in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_lc_https = lc_tindex_master['https'].loc[lc_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
 
    else:
        print(f"No filtered ATL08 for tile {INPUT_TILE_NUM}. Moving on.." )
        continue
    
    # iters is the number of iterations of RH model fits; For no uncertainties set iters = 1 for quick mapping / testing.
    # anything >1 will be exhaustive until uncertainties constrain or iters becomes 250. Will take a long time.
        
    # ppside is the number of subtile splits. 2=4 subtiles, 3=9 subtiles
    # increasing ppside reduces memory, increases length to completion
    
    # minDOY is the min possible DOY to search for tile training data. The default is set to May 1
    # If sufficient data are found in growing season, minDOY will not be used
    
    # maxDOY is the max possible DOY to search for tile training data. The default is Sept 30.
    
    # max_sol_el is the max solar elevation possible to search. The default is 0
    
    # expand_training as TRUE sets iterative searching for training data up to min_n, starting with solar_el<0 and growing season and then:
    # searches solar_el up to max_sol_el then
    # searches iteratively one month later after growing season until maxDOY then
    # searches iteratively one month earlier before growing season until minDOY
    # if expand_training=FALSE the growing season sol_el<0 will be taken, and if none, no data will return
    
    # min_n is the tile training number desired, that expand_training will search until it is fulfilled (or the max available)
    
    #local_train_perc is the percent tile training data vs. boreal-wide training data. 100 = all local, 0 = all boreal wide
        
    in_param_dict = {
                                    'in_atl08_fn': 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/ATL08_filt/c2020/tile_atl08/ATL08_filt_tindex_master.csv',
                                    #'in_atl08_fn': f"input/{os.path.basename(in_atl08_https)}",
                                    'in_topo_fn': f"input/{os.path.basename(in_topo_https)}",
                                    'in_landsat_fn': f"input/{os.path.basename(in_landsat_https)}",
                                    'in_lc_fn': f"input/{os.path.basename(in_lc_https)}",
                                    'in_atl08_fn_url': in_atl08_https,
                                    'in_topo_fn_url': in_topo_https,
                                    'in_landsat_fn_url': in_landsat_https,
                                    'in_lc_fn_url': in_lc_https,
                                    'DO_SLOPE_VALID_MASK': 'TRUE',
                                    'in_atl08_sample_fn':f"input/{os.path.basename(train_data_https)}",
                                    'in_atl08_sample_url':train_data_https,
                                    'in_tile_num': INPUT_TILE_NUM,
                                    'in_tile_fn_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
                                    'in_tile_fn': 'input/boreal_tiles_v003.gpkg',
                                    'in_tile_field': 'tile_num',
                                    'iters': 30,
                                    'ppside': 2,
                                    'minDOY':130,
                                    'maxDOY':250,
                                    'max_sol_el':5,
                                    'expand_training': 'TRUE',
                                    'local_train_perc': 100,
                                    'min_n': 5000, 
                                    'boreal_vect': 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/shared/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson',
                                    'boreal_vect_fn': 'input/wwf_circumboreal_Dissolve_reprj.geojson',
                                    'predict_var': 'AGB'
                }
    
    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='map_boreal_2022_rh_noground_v4',#, ht_boreal_2022_rh_noground_v1 ,'map_boreal_2022_rh_ground_v1','map_boreal_2022_rh_noground_v1',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )

    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_TILE_NUM
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE

    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 2, 25, 50, 100, 500, 1000, 1500, 2000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 
        #local_path

# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df

# of input tiles for DPS: 15
DPS run #: 1	| tile num: 1216	| submit status: success	| job id: c86e5c37-e31b-4101-ba42-b3405e33b846
DPS run #: 2	| tile num: 1316	| submit status: success	| job id: 69c3aafb-6b65-423c-b516-732242f06414
Current time:	202302152149
CPU times: user 206 ms, sys: 14 ms, total: 220 ms
Wall time: 2.79 s


,status,http_status_code,job_id,dps_num,tile_num,submit_time,dbs_job_hour,algo_id,user,worker_type
0,success,200,c86e5c37-e31b-4101-ba42-b3405e33b846,1,1216,2023-02-15 21:49:10.681590,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,69c3aafb-6b65-423c-b516-732242f06414,2,1316,2023-02-15 21:49:10.736493,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,0d97edf3-c0aa-4eb1-ac23-7ad0e0c7d6ee,3,26341,2023-02-15 21:49:10.815686,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,aeb05c10-05de-40d7-8a3d-db961e848037,4,1318,2023-02-15 21:49:10.895460,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,f21b310b-c193-4b76-8a7c-80529f030030,5,39464,2023-02-15 21:49:11.103311,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,cc999e4f-7daa-479d-9dc5-ccbd42622dc6,6,24394,2023-02-15 21:49:11.372262,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,9abd6c25-a005-4abb-ad41-c42b654ae81e,7,36238,2023-02-15 21:49:11.592969,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,54d60e0f-e8d3-4d36-835d-8f77f5620189,8,34607,2023-02-15 21:49:11.793211,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,418bc4cf-e3a2-43a7-b9e4-b6ac51a37c08,9,39630,2023-02-15 21:49:11.967947,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb
0,success,200,dadf2c6b-3967-43f3-b97e-ab953333198f,10,41809,2023-02-15 21:49:12.146664,21,run_boreal_biomass_quick_v2_ubuntu,lduncanson,maap-dps-worker-32gb


## Assess DPS Results

In [19]:
%%time
#submit_results_df = pd.read_csv('/projects/my-public-bucket/DPS_run_boreal_biomass_quick_v2_submission_results_68_202302150220.csv')
submit_results_df = pd.concat(submit_results_df_list)

def BUILD_TABLE_JOBSTATUS(submit_results_df):
    import xmltodict
    df = pd.concat([pd.DataFrame(xmltodict.parse(maap.getJobStatus(job_id).content)).transpose() for job_id in submit_results_df.job_id.to_list()])
    return df

job_status_df = BUILD_TABLE_JOBSTATUS(submit_results_df)
#print(job_status_df.head())

num_jobs = submit_results_df.shape[0]
z = submit_results_df.merge(job_status_df, how='left', left_on='job_id',  right_on='wps:JobID')

print(f'Count total jobs:\t{num_jobs}')
print(f"Count pending jobs:\t{z[z['wps:Status'] =='Accepted'].shape[0]}")
print(f"Count running jobs:\t{z[z['wps:Status'] =='Running'].shape[0]}")
print(f"Count succeeded jobs:\t{z[z['wps:Status'] =='Succeeded'].shape[0]}")
print(f"Count failed jobs:\t{z[z['wps:Status'] =='Failed'].shape[0]}")
print(f"% of failed jobs:\t{round(z[z['wps:Status'] =='Failed'].shape[0] / ( z[z['wps:Status'] =='Failed'].shape[0] + z[z['wps:Status'] =='Succeeded'].shape[0] ), 4) * 100}")

Count total jobs:	15
Count pending jobs:	0
Count running jobs:	7
Count succeeded jobs:	0
Count failed jobs:	8
% of failed jobs:	100.0
CPU times: user 177 ms, sys: 14.1 ms, total: 191 ms
Wall time: 394 ms


In [41]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Failed'].iloc[0].job_id).content)

{'wps:Result': {'@xmlns:ows': 'http://www.opengis.net/ows/2.0',
  '@xmlns:schemaLocation': 'http://schemas.opengis.net/wps/2.0/wps.xsd',
  '@xmlns:wps': 'http://www.opengis.net/wps/2.0',
  '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  'wps:JobID': '9b9e1479-faf2-4093-b1ca-03fb2dbe1bed',
  'wps:Output': {'@id': 'traceback',
   'wps:Data': 'Traceback (most recent call last):\n  File "/home/ops/verdi/ops/hysds-0.3.11/hysds/job_worker.py", line 1126, in run_job\n    raise RuntimeError("Got non-zero exit code: {}".format(status))\nRuntimeError: Got non-zero exit code: 255'}}}

In [136]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Failed'].iloc[20].job_id).content)

{'wps:Result': {'@xmlns:ows': 'http://www.opengis.net/ows/2.0',
  '@xmlns:schemaLocation': 'http://schemas.opengis.net/wps/2.0/wps.xsd',
  '@xmlns:wps': 'http://www.opengis.net/wps/2.0',
  '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  'wps:JobID': '167d1e36-6d6e-4d4c-9549-b02dae4d038e',
  'wps:Output': {'@id': 'traceback',
   'wps:Data': 'Traceback (most recent call last):\n  File "/home/ops/verdi/ops/hysds-0.3.11/hysds/job_worker.py", line 1085, in run_job\n    monitoredRunner.join()\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/process.py", line 148, in join\n    res = self._popen.wait(timeout)\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/popen_fork.py", line 57, in wait\n    return self.poll(os.WNOHANG if timeout == 0.0 else 0)\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/popen_fork.py", line 33, in poll\n    pid, sts = os.waitpid(self.pid, flag)\n  File "/home/ops/verdi/lib/python3.7/site-packages/billiard/pool.py", l

## After successful DPS run, create AGB tindex

In [6]:
## create AGB tindex 
month_dir_str = '02'
alg_name_str = 'run_boreal_biomass_quick_v2_ubuntu'
maap_version_str = 'map_boreal_2022_rh_noground_v4'
#index_out_dir = os.path.join('/projects/my-private-bucket/dps_output/run_boreal_biomass_qiuck_v2_ubuntu/map_boreal_2022_v3/2022/', month_dir_str)
index_out_dir = '/projects/my-public-bucket/DPS_tile_lists/2023/'
!python /projects/icesat2_boreal/lib/build_tindex_master.py -t 'AGB' -alg_name $alg_name_str --maap_version $maap_version_str -y 2023 -m $month_dir_str -o $index_out_dir

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string

Building a list of tiles:
MAAP version:		map_boreal_2022_rh_noground_v4
Type:		AGB
Year:		2023
Month:		['02']
Days:		1-31

Output dir:  /projects/my-public-bucket/DPS_tile_lists/2023/
                                             s3_path  ...                                    file
0  s3://maap-ops-workspace/lduncanson/dps_output/...  ...  boreal_agb_202302031675450331_0225.tif
1  s3://maap-ops-workspace/lduncanson/dps_output/...  ...  boreal_agb_202302031675450345_0177.tif
2  s3://maap-ops-workspace/lduncanson/dps_output/...  ...  boreal_agb_202302031675450462_0043.tif
3  s3://maap-ops-workspace/lduncanson/dps_output/...  ...  boreal_agb_202302031675450562_0054.tif
4  s3://maap-ops-works

In [14]:
TEST_DPS  = True

if TEST_DPS:
    
    if True:
        #!python /projects/icesat2_boreal/lib/build_tindex_master_v2.py -t AGB -y 2022 -m $month_dir_str -o $index_out_dir
        t = pd.read_csv(os.path.join(index_out_dir,'AGB_tindex_master.csv'))
        COMPLETED_TILES = t.tile_num.to_list()
        NEED_TILES = list(set(DPS_INPUT_TILE_NUM_LIST) - set(COMPLETED_TILES))

        #print(NEED_TILES)
        DPS_INPUT_TILE_NUM_LIST = NEED_TILES
    
else:
    DPS_INPUT_TILE_NUM_LIST = INPUT_TILE_NUM_LIST
   

#DPS_INPUT_TILE_NUM_LIST = [248, 273, 272, 271, 324]
print(f"# of tiles to run: {len(DPS_INPUT_TILE_NUM_LIST)}\n", DPS_INPUT_TILE_NUM_LIST)


# of tiles to run: 15
 [1216, 1316, 26341, 1318, 39464, 24394, 36238, 34607, 39630, 41809, 35059, 35060, 39832, 26554, 42558]


## Re-run Failed Tiles in DPS

In [18]:
%%time
submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):

    DPS_num = i+1
    IDENTIFIER = 'run_boreal_biomass_quick_v2'
    ALGO_ID = f'{IDENTIFIER}_ubuntu'
    USER = 'lduncanson'
    WORKER_TYPE = 'maap-dps-worker-16gb'
    
    # Maybe we're using an input tile list that isnt built directly from an ATL08_filt CSV
    # so, check if we have filtered ATL08 for this tile:
    if INPUT_TILE_NUM in atl08_filt_tindex_master['tile_num'].to_list():
        
        # Get the s3 paths of the corresponding input filenames with an input tile_num
        in_atl08_https =  atl08_filt_tindex_master['https'].loc[atl08_filt_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_topo_https = topo_tindex_master['https'].loc[topo_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_landsat_https = landsat_tindex_master['https'].loc[landsat_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
        in_lc_https = lc_tindex_master['https'].loc[lc_tindex_master['tile_num'] == INPUT_TILE_NUM].tolist()[0]
 
    else:
        print(f"No filtered ATL08 for tile {INPUT_TILE_NUM}. Moving on.." )
        continue
    
    # iters is the number of iterations of RH model fits; set to low (2-5) to test, 30-50 for production
    # 30 yields run times ~4 hours
        
    # ppside is the number of subtile splits. 2=4 subtiles, 3=9 subtiles
    # increasing ppside reduces memory, increases length to completion
    
    # minDOY is the min possible DOY to search for tile training data. The default is set to May 1
    # If sufficient data are found in growing season, minDOY will not be used
    
    # maxDOY is the max possible DOY to search for tile training data. The default is Sept 30.
    
    # max_sol_el is the max solar elevation possible to search. The default is 0
    
    # expand_training as TRUE sets iterative searching for training data up to min_n, starting with solar_el<0 and growing season and then:
    # searches solar_el up to max_sol_el then
    # searches iteratively one month later after growing season until maxDOY then
    # searches iteratively one month earlier before growing season until minDOY
    # if expand_training=FALSE the growing season sol_el<0 will be taken, and if none, no data will return
    
    # min_n is the tile training number desired, that expand_training will search until it is fulfilled (or the max available)
    
    #local_train_perc is the percent tile training data vs. boreal-wide training data. 100 = all local, 0 = all boreal wide
    
    
    in_param_dict = {
                                    'in_atl08_fn': 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/fall2022/ATL08_filt_tindex_master.csv',
                                    #'in_atl08_fn': f"input/{os.path.basename(in_atl08_https)}",
                                    'in_topo_fn': f"input/{os.path.basename(in_topo_https)}",
                                    'in_landsat_fn': f"input/{os.path.basename(in_landsat_https)}",
                                    'in_lc_fn': f"input/{os.path.basename(in_lc_https)}",
                                    'in_atl08_fn_url': in_atl08_https,
                                    'in_topo_fn_url': in_topo_https,
                                    'in_landsat_fn_url': in_landsat_https,
                                    'in_lc_fn_url': in_lc_https,
                                    'DO_SLOPE_VALID_MASK': 'TRUE',
                                    'in_atl08_sample_fn':f"input/{os.path.basename(train_data_https)}",
                                    'in_atl08_sample_url':train_data_https,
                                    'in_tile_num': INPUT_TILE_NUM,
                                    'in_tile_fn_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
                                    'in_tile_fn': 'input/boreal_tiles_v003.gpkg',
                                    'in_tile_field': 'tile_num',
                                    'iters': 1,
                                    'ppside': 2,
                                    'minDOY':130,
                                    'maxDOY':250,
                                    'max_sol_el':5,
                                    'expand_training': 'TRUE',
                                    'local_train_perc': 100,
                                    'min_n': 5000, 
                                    'boreal_vect': 'https://maap-ops-workspace.s3.us-west-2.amazonaws.com/shared/nathanmthomas/analyze_agb/input_zones/wwf_circumboreal_Dissolve_reprj.geojson',
                                    'boreal_vect_fn': 'input/wwf_circumboreal_Dissolve_reprj.geojson'
                }
    
    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version='map_boreal_2022_v3',
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_param_dict
        )

    # Build a dataframe of submission details
    submit_result['dps_num'] = DPS_num
    submit_result['tile_num'] = INPUT_TILE_NUM
    submit_result['submit_time'] = datetime.datetime.now()
    submit_result['dbs_job_hour'] =datetime.datetime.now().hour
    submit_result['algo_id'] = ALGO_ID
    submit_result['user'] = USER
    submit_result['worker_type'] = WORKER_TYPE

    # Append to a list of data frames of submission results
    submit_results_df_list.append(pd.DataFrame([submit_result]))
    
    if DPS_num in [1, 2, 25, 50, 100, 500, 1000, 1500, 2000, 3000, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result['status']}\t| job id: {submit_result['job_id']}") 
        #local_path

# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/DPS_{IDENTIFIER}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df

# of input tiles for DPS: 219
DPS run #: 1	| tile num: 1539	| submit status: success	| job id: 24a8117e-1e63-4762-8007-a71545ae36e2
DPS run #: 2	| tile num: 3078	| submit status: success	| job id: ba3efa1c-f4c9-4f86-80a0-e17b5a1d2c07
DPS run #: 25	| tile num: 3633	| submit status: success	| job id: a5a7b136-c7bc-4ddd-a28f-aca8a01f96b9
DPS run #: 50	| tile num: 3178	| submit status: success	| job id: 9a772e83-2864-495b-8758-f79780c04abf
DPS run #: 100	| tile num: 2265	| submit status: success	| job id: 695d01d8-d6dd-4bab-922f-f0dd2e15e30f
Current time:	202211210208


ImportError: cannot import name 'CompressionOptions' from 'pandas._typing' (/opt/conda/lib/python3.7/site-packages/pandas/_typing.py)

## Re-Assess Re-submission
Continue with last few steps until all jobs successful: create tindex, create new tile list, re-run missing tiles in DPS, re-assess

In [36]:
%%time
#submit_results_df = pd.read_csv('/projects/my-public-bucket/DPS_run_boreal_biomass_quick_v2_submission_results_2494_202211132259.csv')
submit_results_df = pd.concat(submit_results_df_list)

def BUILD_TABLE_JOBSTATUS(submit_results_df):
    import xmltodict
    df = pd.concat([pd.DataFrame(xmltodict.parse(maap.getJobStatus(job_id).content)).transpose() for job_id in submit_results_df.job_id.to_list()])
    return df

job_status_df = BUILD_TABLE_JOBSTATUS(submit_results_df)
#print(job_status_df.head())

num_jobs = submit_results_df.shape[0]
z = submit_results_df.merge(job_status_df, how='left', left_on='job_id',  right_on='wps:JobID')

print(f'Count total jobs:\t{num_jobs}')
print(f"Count pending jobs:\t{z[z['wps:Status'] =='Accepted'].shape[0]}")
print(f"Count running jobs:\t{z[z['wps:Status'] =='Running'].shape[0]}")
print(f"Count succeeded jobs:\t{z[z['wps:Status'] =='Succeeded'].shape[0]}")
print(f"Count failed jobs:\t{z[z['wps:Status'] =='Failed'].shape[0]}")
print(f"% of failed jobs:\t{round(z[z['wps:Status'] =='Failed'].shape[0] / ( z[z['wps:Status'] =='Failed'].shape[0] + z[z['wps:Status'] =='Succeeded'].shape[0] ), 4) * 100}")

Count total jobs:	219
Count pending jobs:	0
Count running jobs:	70
Count succeeded jobs:	15
Count failed jobs:	134
% of failed jobs:	89.92999999999999
CPU times: user 2.24 s, sys: 197 ms, total: 2.44 s
Wall time: 5.08 s


## Get other lists just of missing tiles (various ways)

In [22]:
need_tindex_master = pd.read_csv('/projects/my-public-bucket/DPS_tile_lists/Need_AGB_tindex_master.csv')
print(len(need_tindex_master))

INPUT_TILE_NUM_LIST = need_tindex_master.tile_num.tolist()

# Remove duplicate tile nums
INPUT_TILE_NUM_LIST = list(set(INPUT_TILE_NUM_LIST))

print(len(INPUT_TILE_NUM_LIST))

INPUT_TILE_NUM_LIST_NEED = INPUT_TILE_NUM_LIST

1
1


In [9]:
weird_tiles = [4304, 4305, 4221, 4220, 1785, 1718, 1720, 1661, 1257, 1318, 1317]
werid_tiles = [1255, 1196, 949, 1062, 1063, 1005, 950, 1004]
INPUT_TILE_NUM_LIST = weird_tiles
len(INPUT_TILE_NUM_LIST)

11

In [10]:
# Get all boreal tiles
ATL08_filt_tindex_master =   pd.read_csv('s3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/ATL08_filt_tindex_master.csv')
boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_grid_albers90k_gpkg.gpkg'
boreal_tile_index = geopandas.read_file(boreal_tile_index_path)
boreal_tile_index.rename(columns={"layer":"tile_num"}, inplace=True)
boreal_tile_index["tile_num"] = boreal_tile_index["tile_num"].astype(int)

bad_tiles = [3540,3634,3728,3823,3916,4004] #Dropping the tiles near antimeridian that reproject poorly.
select_needs = [3360,2994,3190,2840,3012,3014,3017,2932,1261,1263,1264,988,978,794, 380,378,411,821,861,
                812,765,764,1308,1302,1469,1406,2495,2883,2965,3321,3509,3510,3327,3335,2976,2906,2907,2894,2814,4253,4293,4403,4440,4408,4372,4477,3986]

# Remove bad tiles
boreal_tile_index = boreal_tile_index[~boreal_tile_index['tile_num'].isin(bad_tiles)]
    
#print(boreal_tile_index.head())
tile_matches_select_needs = boreal_tile_index.merge(ATL08_filt_tindex_master[ATL08_filt_tindex_master['tile_num'].isin(select_needs)], how='right', on='tile_num')
print(len(tile_matches_select_needs))
select_needs_filt = print([t for t in tile_matches_select_needs.tile_num])

45
[978, 988, 978, 988, 1263, 1261, 1264, 794, 2932, 3190, 3012, 3014, 3360, 3017, 378, 765, 812, 821, 764, 861, 1302, 1308, 1469, 2894, 2883, 2965, 2976, 3327, 3321, 3510, 3509, 4372, 4440, 4477, 2994, 1406, 2840, 411, 380, 3335, 2495, 4403, 2906, 2907, 2814]


In [32]:
#select_needs_filt = [978, 988, 978, 988, 1263, 1261, 1264, 794, 2932, 3190, 3012, 3014, 3360, 3017, 378, 765, 812, 821, 764, 861, 1302, 1308, 1469, 2894, 2883, 2965, 2976, 3327, 3321, 3510, 3509, 4372, 4440, 4477, 2994, 1406, 2840, 411, 380, 3335, 2495, 4403, 2906, 2907, 2814]

print(len(select_needs_filt))

import numpy as np
tile_nums_missing_but_wont_run = np.setdiff1d(select_needs, select_needs_filt)
print(len(tile_nums_missing_but_wont_run))
print(tile_nums_missing_but_wont_run)

tile_nums_missing_periphery = np.setdiff1d(INPUT_TILE_NUM_LIST_NEED, select_needs_filt)
print(len(tile_nums_missing_periphery))
print(tile_nums_missing_periphery)
INPUT_TILE_NUM_LIST = tile_nums_missing_periphery
len(INPUT_TILE_NUM_LIST)    

45
4
[3986 4253 4293 4408]
70
[  65  106  127  250  361  417  493  566  568  618  741  753  786  798
  801  830  848  876  877  927 1273 1287 1318 1402 1438 1439 1440 1604
 1848 1850 1852 1936 2425 2426 2952 3124 3355 3540 3542 3634 3728 3823
 3845 3846 3895 3916 4004 4007 4018 4083 4176 4204 4250 4256 4307 4325
 4342 4395 4417 4444 4450 4451 4463 4467 4468 4483 4499 4510 4527 4534]


70

In [18]:
INPUT_TILE_NUM_LIST = [4440,4372,4477]

## Send output cog.tif tiles somewhere

In [12]:
import glob
out_files = glob.glob("/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/**/*cog.tif", recursive=True)

In [13]:
print(out_files)

['/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/02/23/49/35/272239/boreal_agb_20220202_0200_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/01/42/37/290043/boreal_agb_20220203_0224_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/01/52/46/430279/boreal_agb_20220203_0387_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/02/21/50/348844/boreal_agb_20220203_0386_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/01/42/237812/boreal_agb_20220203_0199_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/06/47/336472/boreal_agb_20220203_0223_cog.tif', '/projects/my-private-bucket/dps_output/run_boreal_biomass_ubuntu/master/2022/02/baseline_run/03/03/08/52/15295

In [ ]:
TEST_SUBDIR = 'add_solar'
OUTDIR = f'/projects/my-private-bucket/dps_output/run_boreal_biomass_v2_ubuntu/master/2022/{TEST_SUBDIR}'
!python /projects/icesat2_boreal/lib/build_tindex_master.py -y 2022 -m $TEST_SUBDIR -o $OUTDIR